In [1]:
# %load_ext autoreload
# %autoreload 2
# %load_ext wurlitzer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

import xobjects as xo
import xpart as xp
import xcoll as xc

In [2]:
def _create_geometry_kernel():
    src_poly = [xc._pkg_root / 'scattering_routines' / 'geometry' / 'polygon.h',
                xc._pkg_root / 'scattering_routines' / 'geometry' / 'crystal.h']
    kernels_poly = {
        'check_poly': xo.Kernel(
                c_name='get_s_of_first_crossing_with_polygon',
                args=[
                    xo.Arg(xo.Float64, pointer=False, name='part_x'),
                    xo.Arg(xo.Float64, pointer=False, name='part_tan'),
                    xo.Arg(xo.Float64, pointer=True, name='poly_s'),
                    xo.Arg(xo.Float64, pointer=True, name='poly_x'),
                    xo.Arg(xo.Int8, name='num_polys'),
                    xo.Arg(xo.Int8, name='is_closed')
                ],
                ret=xo.Arg(xo.Float64, pointer=False, name='s')),
        'check_open_poly': xo.Kernel(
                c_name='get_s_of_first_crossing_with_open_polygon',
                args=[
                    xo.Arg(xo.Float64, pointer=False, name='part_x'),
                    xo.Arg(xo.Float64, pointer=False, name='part_tan'),
                    xo.Arg(xo.Float64, pointer=True, name='poly_s'),
                    xo.Arg(xo.Float64, pointer=True, name='poly_x'),
                    xo.Arg(xo.Int8, name='num_polys'),
                    xo.Arg(xo.Float64, name='tan_tilt'),
                    xo.Arg(xo.Int8, name='side')
                ],
                ret=xo.Arg(xo.Float64, pointer=False, name='s')),
        'check_cry': xo.Kernel(
                c_name='get_s_of_first_crossing_with_crystal',
                args=[
                    xo.Arg(xo.Float64, pointer=False, name='part_x'),
                    xo.Arg(xo.Float64, pointer=False, name='part_tan'),
                    xo.Arg(xo.Float64, pointer=False, name='R'),
                    xo.Arg(xo.Float64, pointer=False, name='width'),
                    xo.Arg(xo.Float64, pointer=False, name='length'),
                    xo.Arg(xo.Float64, pointer=False, name='jaw_U'),
                    xo.Arg(xo.Float64, pointer=False, name='sin_tilt'),
                    xo.Arg(xo.Float64, pointer=False, name='cos_tilt')
                ],
                ret=xo.Arg(xo.Float64, pointer=False, name='s'))
    }
    context = xo.ContextCpu()
    context.add_kernels(sources=src_poly, kernels=kernels_poly)
    return context.kernels

def _generate_polygon_points(num_poly, tilt_L=0, tilt_R=0):
    with open(xc._pkg_root.parent / "tests" / "data" / "geometry.json", "r") as fp:
        rans = json.load(fp)['rans']
    len_between = num_poly-4
    between = [[(i+1)/(len_between+1) + rans[i]*0.15-0.075, rans[i+len_between]*0.15+0.025]
               for i in range(len_between)]
    poly_L = [[0,0.4],  [0,0.1],  *between, [1,0.1],  [1,0.4]]
    between = [[(i+1)/(len_between+1) + rans[2*len_between+i]*0.15-0.075, -rans[3*len_between+i]*0.15-0.025]
               for i in range(len_between)]
    poly_R = [[0,-0.4], [0,-0.1], *between, [1,-0.1], [1,-0.4]]
    cos_L = np.cos(np.deg2rad(-tilt_L))
    sin_L = np.sin(np.deg2rad(-tilt_L))
    cos_R = np.cos(np.deg2rad(-tilt_R))
    sin_R = np.sin(np.deg2rad(-tilt_R))
    poly_s_L = np.array([(s-0.5)*cos_L  + (x-0.1)*sin_L + 0.5 for s,x in poly_L], dtype=np.float64)
    poly_x_L = np.array([-(s-0.5)*sin_L + (x-0.1)*cos_L + 0.1 for s,x in poly_L], dtype=np.float64)
    poly_s_R = np.array([(s-0.5)*cos_R  + (x+0.1)*sin_R + 0.5 for s,x in poly_R], dtype=np.float64)
    poly_x_R = np.array([-(s-0.5)*sin_R + (x+0.1)*cos_R - 0.1 for s,x in poly_R], dtype=np.float64)
    return poly_s_L, poly_x_L, poly_s_R, poly_x_R

def _generate_crystal_points(R, w, tilt):
    alpha = np.arcsin(1/R)
    poly_s = np.linspace(0, 1, 200)
    poly_x = R + 0.1 - np.sign(R)*np.sqrt(R**2 - poly_s**2)
    if R < 0:
        poly_x += w
    D_s = poly_s[-1] - w/abs(R)
    D_x = poly_x[-1] + w/R*np.sqrt(R**2 - 1)
    poly_s_2 = np.flip(np.linspace(0, D_s, 200))
    poly_x_2 = R + 0.1 - np.sign(R)*np.sqrt((abs(R)-w)**2 - poly_s_2**2)
    if R < 0:
        poly_x_2 += w
    poly_s = np.concatenate([poly_s, poly_s_2, [poly_s[0]]])
    poly_x = np.concatenate([poly_x, poly_x_2, [poly_x[0]]])
    cos_L = np.cos(np.deg2rad(-tilt))
    sin_L = np.sin(np.deg2rad(-tilt))
    d = 0.1 if R > 0 else 0.1 + w
    poly_s_L = np.array([(s-0.5)*cos_L  + (x-0.1)*sin_L + 0.5 for s,x in zip(poly_s, poly_x)], dtype=np.float64)
    poly_x_L = np.array([-(s-0.5)*sin_L + (x-0.1)*cos_L + 0.1 for s,x in zip(poly_s, poly_x)], dtype=np.float64)
    poly_s_R = np.array([(s-0.5)*cos_L  + (x-0.1)*sin_L + 0.5 for s,x in zip([0, 1, 1, 0, 0], [d, d, D_x, D_x, d])], dtype=np.float64)
    poly_x_R = np.array([-(s-0.5)*sin_L + (x-0.1)*cos_L + 0.1 for s,x in zip([0, 1, 1, 0, 0], [d, d, D_x, D_x, d])], dtype=np.float64)
    return poly_s_L, poly_x_L, poly_s_R, poly_x_R

In [3]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

kernels = _create_geometry_kernel()

@interact(tilt_L=widgets.IntSlider(min=-89, max=89, step=1, value=0),
          tilt_R=widgets.IntSlider(min=-89, max=89, step=1, value=0),
          part_x_cm=widgets.IntSlider(min=-120, max=120, step=1, value=0),
          part_theta=widgets.IntSlider(min=-89, max=89, step=1, value=0),
          num_poly=widgets.IntSlider(min=4, max=8, step=1, value=4),
          is_open=widgets.Checkbox(value=True, description='is_open'))
def check(tilt_L, tilt_R, part_x_cm, part_theta, num_poly, is_open):
    part_x = part_x_cm/100
    part_tan = np.tan(np.deg2rad(part_theta))
    p_s_L, p_x_L, p_s_R, p_x_R = _generate_polygon_points(num_poly, tilt_L, tilt_R)
    if is_open:
        s_L = kernels.check_open_poly(part_x=part_x, part_tan=part_tan, poly_s=p_s_L[1:-1],
                                      poly_x=p_x_L[1:-1], num_polys=len(p_s_L)-2,
                                      tan_tilt=np.tan(np.deg2rad(tilt_L)), side=1)
        s_R = kernels.check_open_poly(part_x=part_x, part_tan=part_tan, poly_s=p_s_R[1:-1],
                                      poly_x=p_x_R[1:-1], num_polys=len(p_s_R)-2,
                                      tan_tilt=np.tan(np.deg2rad(tilt_R)), side=-1)
    else:
        s_L = kernels.check_poly(part_x=part_x, part_tan=part_tan, poly_s=p_s_L,
                                 poly_x=p_x_L, num_polys=len(p_s_L), is_closed=True)
        s_R = kernels.check_poly(part_x=part_x, part_tan=part_tan, poly_s=p_s_R,
                                 poly_x=p_x_R, num_polys=len(p_s_R), is_closed=True)
    hit = 0
    if s_L < 1.e20:
        hit = 1
        s = s_L
    if s_R < 1.e20 and s_R < s_L:
        hit = -1
        s = s_R
    fig, ax = plt.subplots(1, 1, figsize=(8,5.6))
    if is_open:
        ax.plot(p_s_L, p_x_L, 'k-')
        ax.plot(p_s_R, p_x_R, 'k-')
    else:
        ax.plot([*p_s_L, p_s_L[0]], [*p_x_L, p_x_L[0]], 'k-')
        ax.plot([*p_s_R, p_s_R[0]], [*p_x_R, p_x_R[0]], 'k-')
    ax.plot([-0.5,1.5], [part_x-0.5*part_tan,part_x+1.5*part_tan], 'b-')
    ax.set_xlim((-0.5,1.5))
    ax.set_ylim((-0.7,0.7))
    if hit != 0:
        if hit == 1:
            print(f"Hit left  {s}")
        else:
            print(f"Hit right  {s}")
        ax.axvline(s, c='r', ls='--')
    else:
        print("No hit")

FileNotFoundError: [Errno 2] No such file or directory: '/afs/cern.ch/user/c/cmaccani/xsuite/xcoll/xcoll/scattering_routines/geometry/polygon.h'

In [4]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

kernels = _create_geometry_kernel()

@interact(tilt=widgets.IntSlider(min=-89, max=89, step=1, value=0),
          part_x_cm=widgets.IntSlider(min=-120, max=120, step=1, value=0),
          part_theta=widgets.IntSlider(min=-89, max=89, step=1, value=0),
          R=widgets.IntSlider(min=1, max=10, step=1, value=2),
          width_cm=widgets.IntSlider(min=1, max=30, step=1, value=15),
          R_neg=widgets.Checkbox(value=False, description='R_neg'))
def check_cry(tilt, part_x_cm, part_theta, R, width_cm, R_neg):
    part_x = part_x_cm/100
    part_tan = np.tan(np.deg2rad(part_theta))
    width = width_cm/100
    if R_neg:
        R *= -1
    pts_s, pts_x, box_s, box_x = _generate_crystal_points(R, width, tilt)
    s = kernels.check_cry(part_x=part_x, part_tan=part_tan, R=R, width=width,
                          length=1, jaw_U=0.1-0.5*np.sin(np.deg2rad(tilt)),
                          sin_tilt=np.sin(np.deg2rad(tilt)), cos_tilt=np.cos(np.deg2rad(tilt)))
    hit = 0
    if s < 1.e20:
        hit = 1
    fig, ax = plt.subplots(1, 1, figsize=(8,5.6))
    ax.plot(pts_s, pts_x, 'k-')
    ax.plot(box_s, box_x, 'k--')
    ax.plot([-0.5,1.5], [part_x-0.5*part_tan,part_x+1.5*part_tan], 'b-')
    ax.set_xlim((-0.5,1.5))
    ax.set_ylim((-0.7,0.7))
    if hit != 0:
        ax.axvline(s, c='r', ls='--')

FileNotFoundError: [Errno 2] No such file or directory: '/afs/cern.ch/user/c/cmaccani/xsuite/xcoll/xcoll/scattering_routines/geometry/polygon.h'